In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# Load dataset
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)

    # One-hot encoding for categorical columns
    data = pd.get_dummies(data, columns=['month', 'day'], drop_first=True)

    # Separate features and target
    X = data.drop(columns=['area'])
    y = data['area']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train.values, y_test.values

file_path = '/content/forestfires.csv'
X_train, X_test, y_train, y_test = load_and_preprocess_data(file_path)

In [3]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [4]:
# Create DataLoader for batch processing
def create_data_loader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
# Define the MLP regression model
class MLPRegressionModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPRegressionModel, self).__init__()
        layers = []
        in_features = input_size

        for hidden_units in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation_fn)
            in_features = hidden_units

        layers.append(nn.Linear(in_features, 1))  # Output layer for regression
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [6]:
# Train and evaluate the model
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Evaluate
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            predictions.extend(outputs.view(-1).tolist())
            actuals.extend(y_batch.view(-1).tolist())

    mse = mean_squared_error(actuals, predictions)
    print(f"Mean Squared Error: {mse:.4f}")
    return mse

In [7]:
# Configuration variables
input_size = X_train.shape[1]
hidden_layer_configs = [[4], [8], [16], [32], [64]]  # Example configurations
activation_functions = {'Linear': nn.Identity(), 'Sigmoid': nn.Sigmoid(), 'ReLU': nn.ReLU(), 'Softmax': nn.Softmax(dim=1), 'Tanh': nn.Tanh()}
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]
epochs_list = [1, 10, 25, 50, 100, 250]

In [8]:
# Experiment: Hidden Layer Comparison
results_hidden_layers = []
for hidden_layers in hidden_layer_configs:
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size=32)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size=32)

    model = MLPRegressionModel(input_size, hidden_layers, nn.ReLU())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    mse = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=50)

    results_hidden_layers.append({
        'hidden_layers': hidden_layers,
        'mse': mse
    })

Epoch [1/50], Loss: 2189.3948
Epoch [2/50], Loss: 2201.1967
Epoch [3/50], Loss: 2169.8288
Epoch [4/50], Loss: 2143.6415
Epoch [5/50], Loss: 2133.8825
Epoch [6/50], Loss: 2114.7646
Epoch [7/50], Loss: 2092.9977
Epoch [8/50], Loss: 2080.0424
Epoch [9/50], Loss: 2067.1284
Epoch [10/50], Loss: 2063.8888
Epoch [11/50], Loss: 2208.7880
Epoch [12/50], Loss: 2053.7382
Epoch [13/50], Loss: 2051.4999
Epoch [14/50], Loss: 2049.8351
Epoch [15/50], Loss: 2050.4680
Epoch [16/50], Loss: 2047.9937
Epoch [17/50], Loss: 2065.0326
Epoch [18/50], Loss: 2056.3279
Epoch [19/50], Loss: 2046.4641
Epoch [20/50], Loss: 2045.4589
Epoch [21/50], Loss: 2049.9219
Epoch [22/50], Loss: 2045.8998
Epoch [23/50], Loss: 2043.4878
Epoch [24/50], Loss: 2042.9164
Epoch [25/50], Loss: 2055.0835
Epoch [26/50], Loss: 2042.5762
Epoch [27/50], Loss: 2043.6515
Epoch [28/50], Loss: 2046.6125
Epoch [29/50], Loss: 2042.0596
Epoch [30/50], Loss: 2048.8635
Epoch [31/50], Loss: 2039.7119
Epoch [32/50], Loss: 2039.6179
Epoch [33/50], Lo

In [9]:
# Experiment: Activation Function Comparison
results_activation = []
for activation_name, activation_fn in activation_functions.items():
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size=32)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size=32)

    model = MLPRegressionModel(input_size, [16], activation_fn)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    mse = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=50)

    results_activation.append({
        'activation': activation_name,
        'mse': mse
    })

Epoch [1/50], Loss: 2291.4622
Epoch [2/50], Loss: 2097.4260
Epoch [3/50], Loss: 2081.5678
Epoch [4/50], Loss: 2055.7991
Epoch [5/50], Loss: 2058.2922
Epoch [6/50], Loss: 2056.4771
Epoch [7/50], Loss: 2057.9917
Epoch [8/50], Loss: 2045.2653
Epoch [9/50], Loss: 2044.5350
Epoch [10/50], Loss: 2195.0352
Epoch [11/50], Loss: 2037.3543
Epoch [12/50], Loss: 2052.0806
Epoch [13/50], Loss: 2033.2155
Epoch [14/50], Loss: 2032.9396
Epoch [15/50], Loss: 2163.1381
Epoch [16/50], Loss: 2038.3220
Epoch [17/50], Loss: 2159.6360
Epoch [18/50], Loss: 2033.1415
Epoch [19/50], Loss: 2023.4155
Epoch [20/50], Loss: 2037.8966
Epoch [21/50], Loss: 2022.2406
Epoch [22/50], Loss: 2031.1671
Epoch [23/50], Loss: 2024.9735
Epoch [24/50], Loss: 2019.3964
Epoch [25/50], Loss: 2049.2796
Epoch [26/50], Loss: 2018.7209
Epoch [27/50], Loss: 2025.6196
Epoch [28/50], Loss: 2015.7478
Epoch [29/50], Loss: 2019.8346
Epoch [30/50], Loss: 2016.9426
Epoch [31/50], Loss: 2010.3800
Epoch [32/50], Loss: 2013.6294
Epoch [33/50], Lo

In [10]:
# Experiment: Learning Rate Comparison
results_learning_rate = []
for lr in learning_rates:
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size=32)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size=32)

    model = MLPRegressionModel(input_size, [16], nn.ReLU())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    mse = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=50)

    results_learning_rate.append({
        'learning_rate': lr,
        'mse': mse
    })

Epoch [1/50], Loss: 2311936.5222
Epoch [2/50], Loss: 4187.8858
Epoch [3/50], Loss: 4302.1681
Epoch [4/50], Loss: 3863.6001
Epoch [5/50], Loss: 3344.7958
Epoch [6/50], Loss: 2887.8004
Epoch [7/50], Loss: 2576.2280
Epoch [8/50], Loss: 2349.0792
Epoch [9/50], Loss: 2215.6993
Epoch [10/50], Loss: 2142.6526
Epoch [11/50], Loss: 2111.0075
Epoch [12/50], Loss: 2095.3467
Epoch [13/50], Loss: 2076.6900
Epoch [14/50], Loss: 2067.7410
Epoch [15/50], Loss: 2067.5886
Epoch [16/50], Loss: 2071.0108
Epoch [17/50], Loss: 2079.3201
Epoch [18/50], Loss: 2201.2668
Epoch [19/50], Loss: 2086.1123
Epoch [20/50], Loss: 2066.9348
Epoch [21/50], Loss: 2066.2945
Epoch [22/50], Loss: 2067.2308
Epoch [23/50], Loss: 2091.9310
Epoch [24/50], Loss: 2066.4935
Epoch [25/50], Loss: 2066.2342
Epoch [26/50], Loss: 2066.8037
Epoch [27/50], Loss: 2201.8015
Epoch [28/50], Loss: 2066.8842
Epoch [29/50], Loss: 2067.1526
Epoch [30/50], Loss: 2081.8118
Epoch [31/50], Loss: 2071.5023
Epoch [32/50], Loss: 2071.5877
Epoch [33/50],

In [11]:
# Experiment: Batch Size Comparison
results_batch_size = []
for batch_size in batch_sizes:
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size=batch_size)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size=batch_size)

    model = MLPRegressionModel(input_size, [16], nn.ReLU())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    mse = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=50)

    results_batch_size.append({
        'batch_size': batch_size,
        'mse': mse
    })

Epoch [1/50], Loss: 2148.3608
Epoch [2/50], Loss: 2107.9127
Epoch [3/50], Loss: 2056.5546
Epoch [4/50], Loss: 2055.0482
Epoch [5/50], Loss: 2052.2893
Epoch [6/50], Loss: 2048.1825
Epoch [7/50], Loss: 2343.8899
Epoch [8/50], Loss: 2047.5640
Epoch [9/50], Loss: 2079.7422
Epoch [10/50], Loss: 2041.0874
Epoch [11/50], Loss: 2043.7129
Epoch [12/50], Loss: 2042.5846
Epoch [13/50], Loss: 2039.5190
Epoch [14/50], Loss: 2035.6737
Epoch [15/50], Loss: 2033.9278
Epoch [16/50], Loss: 2033.2625
Epoch [17/50], Loss: 2034.8443
Epoch [18/50], Loss: 2028.4874
Epoch [19/50], Loss: 2026.9895
Epoch [20/50], Loss: 2027.6486
Epoch [21/50], Loss: 2319.3294
Epoch [22/50], Loss: 2024.8414
Epoch [23/50], Loss: 2033.2171
Epoch [24/50], Loss: 2030.3977
Epoch [25/50], Loss: 2039.1982
Epoch [26/50], Loss: 2019.7172
Epoch [27/50], Loss: 2024.8045
Epoch [28/50], Loss: 2017.1033
Epoch [29/50], Loss: 2058.3851
Epoch [30/50], Loss: 2016.7563
Epoch [31/50], Loss: 2015.9466
Epoch [32/50], Loss: 2014.7347
Epoch [33/50], Lo

In [12]:
# Experiment: Epoch Comparison
results_epochs = []
for epochs in epochs_list:
    train_loader = create_data_loader(X_train_tensor, y_train_tensor, batch_size=32)
    test_loader = create_data_loader(X_test_tensor, y_test_tensor, batch_size=32)

    model = MLPRegressionModel(input_size, [16], nn.ReLU())
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    mse = train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    results_epochs.append({
        'epochs': epochs,
        'mse': mse
    })

Epoch [1/1], Loss: 2176.8658
Mean Squared Error: 12086.6834
Epoch [1/10], Loss: 2185.4537
Epoch [2/10], Loss: 2156.3938
Epoch [3/10], Loss: 2131.3225
Epoch [4/10], Loss: 2076.0725
Epoch [5/10], Loss: 2058.2880
Epoch [6/10], Loss: 2064.0842
Epoch [7/10], Loss: 2060.0124
Epoch [8/10], Loss: 2073.1674
Epoch [9/10], Loss: 2053.9589
Epoch [10/10], Loss: 2050.0858
Mean Squared Error: 11817.0617
Epoch [1/25], Loss: 2178.7328
Epoch [2/25], Loss: 2152.3079
Epoch [3/25], Loss: 2091.7031
Epoch [4/25], Loss: 2076.9306
Epoch [5/25], Loss: 2061.9851
Epoch [6/25], Loss: 2050.1971
Epoch [7/25], Loss: 2046.6378
Epoch [8/25], Loss: 2047.3226
Epoch [9/25], Loss: 2050.7280
Epoch [10/25], Loss: 2175.4586
Epoch [11/25], Loss: 2043.1458
Epoch [12/25], Loss: 2037.9360
Epoch [13/25], Loss: 2039.8198
Epoch [14/25], Loss: 2039.2078
Epoch [15/25], Loss: 2043.8674
Epoch [16/25], Loss: 2043.3031
Epoch [17/25], Loss: 2033.9929
Epoch [18/25], Loss: 2050.2745
Epoch [19/25], Loss: 2047.8202
Epoch [20/25], Loss: 2032.31

In [13]:
# Display results for each comparison
import pandas as pd

print("Results: Hidden Layer Comparison")
print(pd.DataFrame(results_hidden_layers))

print("\nResults: Activation Function Comparison")
print(pd.DataFrame(results_activation))

print("\nResults: Learning Rate Comparison")
print(pd.DataFrame(results_learning_rate))

print("\nResults: Batch Size Comparison")
print(pd.DataFrame(results_batch_size))

print("\nResults: Epoch Comparison")
print(pd.DataFrame(results_epochs))

Results: Hidden Layer Comparison
  hidden_layers           mse
0           [4]  11798.659960
1           [8]  11767.073436
2          [16]  11775.520470
3          [32]  11763.536288
4          [64]  11709.271209

Results: Activation Function Comparison
  activation           mse
0     Linear  11729.491443
1    Sigmoid  11731.352447
2       ReLU  11732.400958
3    Softmax  11916.532374
4       Tanh  11672.837015

Results: Learning Rate Comparison
   learning_rate           mse
0        10.0000  11864.664954
1         1.0000  11853.044377
2         0.1000  11621.992726
3         0.0100  11783.865333
4         0.0010  11829.224948
5         0.0001  12144.780508

Results: Batch Size Comparison
   batch_size           mse
0          16  11729.955879
1          32  11755.582891
2          64  11787.933751
3         128  11803.403688
4         256  11806.256763
5         512  11807.840175

Results: Epoch Comparison
   epochs           mse
0       1  12086.683398
1      10  11817.061654
2    